In [ ]:
import tkinter as tk

class Othello:
    def __init__(self):
        '''イニシャライズ'''

        self.board = [[0] * 6 for _ in range(6)]  # ゲーム盤の初期化
        self.current_player = 0  # 現在のプレイヤー（0: プレイヤー未選択, 1: 黒, -1: 白）
        self.DIRECTIONS =  [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)] #前後左右斜め
        self.DEPTH = 3 #探索の深さ
        self.WEIGHT = 100 #角や端に対する重み
        self.create_wigets() #ウィジェットの作成
   
    def create_wigets(self):
        '''ウィジェットの作成'''

        self.root = tk.Tk()  # ゲームウィンドウの作成
        # 白黒選択を指示するラベル
        self.selection_label = tk.Label(self.root, text="プレイヤーを選択してください")
        self.selection_label.pack()
        # 自分が黒のボタン
        self.black_button = tk.Button(self.root, text="黒", command=lambda: self.start_game(1))
        self.black_button.pack()
        # 自分が白のボタン
        self.white_button = tk.Button(self.root, text="白", command=lambda: self.start_game(-1))
        self.white_button.pack()
        # 碁盤を表示するキャンバス
        self.canvas = tk.Canvas(self.root, height=400)

        self.score_label = tk.Label(self.root, text="")  # スコアテキストボックス
        self.reset_button = tk.Button(self.root, text="リセット", command=self.reset_game)  # リセットボタン

        self.root.mainloop()

    def start_game(self, player):
        ''' 初期画面かゲーム画面への遷移'''

        # 自分がどっちかを判定
        self.player = player
        # 現在白黒どちらか
        self.current_player = 1 #黒が先攻

        # 最初の画面のボタンとラベルを削除
        self.selection_label.pack_forget()
        self.black_button.pack_forget()
        self.white_button.pack_forget()

        # 碁盤とラベルを表示
        self.canvas.pack()
        self.score_label.pack()

        # 真ん中の4つの石を配置
        self.board[2][2] = self.board[3][3] = 1
        self.board[2][3] = self.board[3][2] = -1

        self.draw_board()  # ゲーム盤の描画
        self.canvas.bind("<Button-1>", self.handle_click)  # クリックイベントのバインド
        # リセットボタンを設置
        self.reset_button.pack()

        # ラベルの情報を更新
        self.update_score_label()

        self.show_valid_moves()  # 有効な手の表示

        # 自分が白を選択した場合はAIに動いてもらう
        if player == -1:
            best_move = self.get_best_move()
            x, y = best_move
            self.make_move(x, y)            

    def reset_game(self):
        '''リセット'''

        # 碁盤、画面を初期状態にする
        self.current_player = 0
        self.score_label.config(text="")
        self.reset_button.pack_forget()
        self.canvas.delete("all")
        self.canvas.unbind("<Button-1>")    

        self.selection_label.pack()
        self.black_button.pack()
        self.white_button.pack()

        self.board = [[0] * 6 for _ in range(6)]

    def end(self):
        '''ゲーム終了時の画面'''
        black_count, white_count = self.count_stones()
        if black_count > white_count:
            self.score_label.config(text="黒の勝利！,黒: {}, 白: {}".format(black_count, white_count))
        elif black_count < white_count:
            self.score_label.config(text="白の勝利！,黒: {}, 白: {}".format(black_count, white_count))
        else:
            self.score_label.config(text="引き分け！,黒: {}, 白: {}".format(black_count, white_count))

    def draw_board(self):
        '''描画更新関数'''
        self.canvas.delete(tk.ALL)  # 既存の描画を削除
        
        # 盤面の描画
        for i in range(6):
            for j in range(6):
                x1, y1 = i * 60, j * 60
                x2, y2 = x1 + 60, y1 + 60
                self.canvas.create_rectangle(x1, y1, x2, y2, fill='green')
                
                # 石の描画
                if self.board[i][j] == 1:
                    self.canvas.create_oval(x1+10, y1+10, x2-10, y2-10, fill='black')
                elif self.board[i][j] == -1:
                    self.canvas.create_oval(x1+10, y1+10, x2-10, y2-10, fill='white')

    def update_score_label(self):
        '''現在の石の個数を表示'''
        black_count, white_count = self.count_stones()
        self.score_label.config(text="黒: {}, 白: {}".format(black_count, white_count))

    def is_valid_move(self, x, y):
        '''その座標が置くことができるか判定'''

        # 碁盤上か判定
        if x < 0 or x > 5 or y < 0 or y > 5:
            return False
        # 石がすでに置いてないか判定
        if self.board[x][y] != 0:
            return False
        
        # ひっくり返す石があるか判定
        for dx, dy in self.DIRECTIONS:
            nx, ny = x + dx, y + dy
            
            if 0 <= nx < 6 and 0 <= ny < 6 and self.board[nx][ny] == -self.current_player:
                while 0 <= nx < 6 and 0 <= ny < 6:
                    if self.board[nx][ny] == 0:
                        break
                    elif self.board[nx][ny] == self.current_player:
                        return True
                    nx += dx
                    ny += dy
        return False

    def get_valid_moves(self):
        ''' 置くことのできるマスを取得'''

        valid_moves = []
        for i in range(6):
            for j in range(6):
                if self.is_valid_move(i, j):
                    valid_moves.append((i, j))
        return valid_moves

    def show_valid_moves(self):
        '''置くことができるマスを青くする'''

        valid_moves = self.get_valid_moves()      
        for move in valid_moves:
            x, y = move
            x1, y1 = x * 60, y * 60
            x2, y2 = x1 + 60, y1 + 60
            self.canvas.create_oval(x1+25, y1+25, x2-25, y2-25, outline='blue')

    def count_stones(self):
        '''石の数を色ごとに数える'''

        black_count = sum(row.count(1) for row in self.board)
        white_count = sum(row.count(-1) for row in self.board)
        return black_count, white_count
    
    def make_move(self, x, y):
        '''石を指定の場所に置き裏返す'''
        # 指定の場所に自分の石を置く
        self.board[x][y] = self.current_player
        
        # ひっくり返す
        for dx, dy in self.DIRECTIONS:
            nx, ny = x + dx, y + dy
            positions_to_flip = []
            
            while 0 <= nx < 6 and 0 <= ny < 6 and self.board[nx][ny] == -self.current_player:
                positions_to_flip.append((nx, ny))
                nx += dx
                ny += dy
            
            if 0 <= nx < 6 and 0 <= ny < 6 and self.board[nx][ny] == self.current_player:
                for position in positions_to_flip:
                    px, py = position
                    self.board[px][py] = self.current_player
        
        # プレーヤーチェンジと描画
        self.next_turn()

    def is_no_move(self):
        # 置く場所があるかないか
        if len(self.get_valid_moves()) == 0:
            return True
        return False

    def next_turn(self):
            # 相手に交代
            self.current_player *= -1

            # 盤面を更新
            self.update_score_label()
            self.draw_board()
            self.show_valid_moves()

    def get_best_move(self):
        '''最良の手を探索し、戻り値はその座標'''

        valid_moves = self.get_valid_moves()
        best_score = float('-inf')
        best_move = None
        
        # 可能な手を全て試して一番期待値が高い手を探索する
        for move in valid_moves:
            x, y = move
            temp_board = [row[:] for row in self.board]  # 盤面のコピー
            temp_player = self.current_player  # プレイヤー情報のコピー
            
            self.make_move(x, y)  # 一時的に手を打つ
            
            score = self.minimax(self.DEPTH, False)  # ミニマックス法で評価値を計算
            # 角の場合加点
            if self.judge_corner(x, y):
                score += 10 * self.WEIGHT
            
            # 端の場合加点
            if self.judge_edge(x, y):
                score += 5 * self.WEIGHT
            
            self.board = temp_board  # 盤面を元に戻す
            self.current_player = temp_player  # プレイヤー情報を元に戻す
            
            if score > best_score:
                best_score = score
                best_move = move
        
        return best_move

    def minimax(self, depth, is_maximizing_player):
        '''ミニマックス法で探索'''
        if depth == 0 or self.is_no_move():
            black_count, white_count = self.count_stones()
            return black_count - white_count

        if is_maximizing_player:
            max_eval = float('-inf')
            valid_moves = self.get_valid_moves()

            for move in valid_moves:
                x, y = move
                temp_board = [row[:] for row in self.board]
                temp_player = self.current_player

                self.make_move(x, y)
                eval = self.minimax(depth - 1, False)
                # 角を取れる場合加点
                if self.judge_corner(x, y):
                    eval += self.WEIGHT
                
                # 端をとれる場合加点
                if self.judge_edge(x, y):
                    eval += self.WEIGHT
                self.board = temp_board
                self.current_player = temp_player

                max_eval = max(max_eval, eval)

            return max_eval
        else:
            min_eval = float('inf')
            valid_moves = self.get_valid_moves()

            for move in valid_moves:
                x, y = move
                temp_board = [row[:] for row in self.board]
                temp_player = self.current_player

                self.make_move(x, y)
                eval = self.minimax(depth - 1, True)
                # 角をとられる場合減点
                if self.judge_corner(x, y):
                    eval -= 10 * self.WEIGHT
                
                # 端をとられる場合減点
                if self.judge_edge(x, y):
                    eval -= 5 * self.WEIGHT
                self.board = temp_board
                self.current_player = temp_player

                min_eval = min(min_eval, eval)

            return min_eval

    def judge_corner(self, x, y):
        '''座標が角かどうか判定'''
        return (x == 0 and y == 0) or (x == 0 and y == 5) or (x == 5 and y == 0) or (x == 5 and y == 5)
    
    def judge_edge(self, x, y):
        '''座標が端かどうか判定'''
        return x==0 or y == 0 or x == 5 or y == 5

    def ai_move(self):
        '''AIの動きを制御'''

        '''
        パス回数は石を置くたびにリセットされる
        '''

        # AIの打つ手がない場合
        if self.is_no_move():
            # 相手に交代
            self.next_turn()

            # さらにパスすすると終了
            if self.is_no_move():
                self.end()
        else:
            # AIの打つ手がある場合

            # 最良の手を打つ
            best_move = self.get_best_move()
            x, y = best_move
            self.make_move(x, y)

            # 自分のターンになり打つ手がない場合
            if self.is_no_move():
                # 相手に交代
                self.next_turn()
                # 再びAIの動き
                self.ai_move()
        
    def handle_click(self, event):
        '''クリックした際に動く関数'''
        # その座標におけるかどうか判定
        x, y = event.x // 60, event.y // 60
        if self.is_valid_move(x, y):
            # 置ける場合はその場所に置き、AIのターンに映る
            self.make_move(x, y)
            self.ai_move()
        else:        
            self.score_label.config(text="無効な手です")

if __name__ == "__main__":
    game = Othello()
